In [1]:
ontology = {
    "types": [
        "model",
        "provider",
        "request",
        "capability",
        "user",
        "price",
        "budget",
        "tokens"
    ],
    "predicates": [
        {"name": "supports", "args": ["?model - model", "?capability - capability"]},
        {"name": "offered-by", "args": ["?model - model", "?provider - provider"]},
        {"name": "requires", "args": ["?request - request", "?capability - capability"]},
        {"name": "can-handle", "args": ["?model - model", "?request - request"]},
        {"name": "model-available", "args": ["?model - model"]},
        {"name": "assigned", "args": ["?request - request", "?model - model"]},
        {"name": "max-tokens", "args": ["?model - model", "?max_tokens - tokens"]},
        {"name": "request-tokens", "args": ["?request - request", "?used_tokens - tokens"]},
        {"name": "price-per-million", "args": ["?model - model", "?provider - provider", "?price - price"]},
        {"name": "user-budget", "args": ["?user - user", "?budget_limit - budget"]},
        {"name": "within-budget", "args": ["?price - price", "?budget_limit - budget"]},
        {"name": "within-token-limit", "args": ["?used_tokens - tokens", "?max_tokens - tokens"]}
    ]
}

In [2]:
from pydantic import BaseModel
from typing import List

class Predicate(BaseModel):
    name: str
    args: List[str]

class Action(BaseModel):
    name: str
    parameters: str
    precondition: List[str]
    effect: List[str]

class Ontology(BaseModel):
    types: List[str]
    predicates: List[Predicate]
    actions: List[Action] = []  # 可选

In [3]:
def build_pddl_domain_file(domain_name: str, ontology: dict, output_path: str = "domain.pddl"):
    def indent_block(items: list[str], indent: int = 6):
        space = " " * indent
        return [f"{space}{item}" for item in items]

    lines = []
    lines.append(f"(define (domain {domain_name})")
    lines.append("  (:requirements :strips :typing)")

    # Types
    lines.append("  (:types")
    lines.append("    " + " ".join(ontology["types"]))
    lines.append("  )")

    # Predicates
    lines.append("  (:predicates")
    for pred in ontology["predicates"]:
        lines.append(f"    ({pred['name']} {' '.join(pred['args'])})")
    lines.append("  )")

    # Actions（如果存在）
    for action in ontology.get("actions", []):
        lines.append(f"  (:action {action['name']}")
        lines.append(f"    :parameters ({action['parameters']})")
        lines.append("    :precondition (and")
        lines.extend(indent_block(action["precondition"], 6))
        lines.append("    )")
        lines.append("    :effect (and")
        lines.extend(indent_block(action["effect"], 6))
        lines.append("    )")
        lines.append("  )")

    lines.append(")")

    with open(output_path, "w") as f:
        f.write("\n".join(lines))
    print(f"domain.pddl saved to {output_path}")

In [4]:
build_pddl_domain_file("openrouter-routing", ontology)

domain.pddl saved to domain.pddl


In [5]:
def build_pddl_problem_file(
    problem_name: str,
    domain_name: str,
    objects: dict,
    init: list[str],
    goal: list[str],
    output_path: str = "problem.pddl"
):
    lines = [f"(define (problem {problem_name})"]
    lines.append(f"  (:domain {domain_name})")

    # Objects
    lines.append("  (:objects")
    for typename, instances in objects.items():
        lines.append(f"    {' '.join(instances)} - {typename}")
    lines.append("  )")

    # Init state
    lines.append("  (:init")
    for fact in init:
        lines.append(f"    {fact}")
    lines.append("  )")

    # Goal state
    lines.append("  (:goal (and")
    for fact in goal:
        lines.append(f"    {fact}")
    lines.append("  ))")

    lines.append(")")

    with open(output_path, "w") as f:
        f.write("\n".join(lines))
    print(f"✅ problem.pddl saved to {output_path}")

In [6]:
objects = {
    "model": ["gpt4", "llama2"],
    "provider": ["openai", "meta"],
    "user": ["user1"],
    "request": ["req1"],
    "capability": ["coding", "multilingual"],
    "tokens": ["tok2000", "tok8000"],
    "price": ["price10"],
    "budget": ["budget20"]
}

init = [
    "(supports gpt4 coding)",
    "(supports gpt4 multilingual)",
    "(supports llama2 multilingual)",
    "(offered-by gpt4 openai)",
    "(offered-by llama2 meta)",
    "(model-available gpt4)",
    "(model-available llama2)",
    "(can-handle gpt4 req1)",
    "(can-handle llama2 req1)",
    "(requires req1 coding)",
    "(request-tokens req1 tok2000)",
    "(max-tokens gpt4 tok8000)",
    "(max-tokens llama2 tok8000)",
    "(within-token-limit tok2000 tok8000)",
    "(price-per-million gpt4 openai price10)",
    "(user-budget user1 budget20)",
    "(within-budget price10 budget20)"
]

goal = [
    "(assigned req1 gpt4)"
]

build_pddl_problem_file("route-request-problem", "openrouter-routing", objects, init, goal)

✅ problem.pddl saved to problem.pddl


In [14]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import OneshotPlanner
from unified_planning.engines import PlanGenerationResultStatus

reader = PDDLReader()
problem = reader.parse_problem("domain.pddl", "problem.pddl")

with OneshotPlanner(name="pyperplan") as planner:
    result = planner.solve(problem)

if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
    print("Plan found:")
    for i, action in enumerate(result.plan.actions):
        print(f"{i+1}. {action}")
else:
    print("No plan.")

Plan found:
1. assign-model(gpt4, req1, tok2000, tok8000, coding, openai, price10, budget20, user1)
